In [39]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import os
import matplotlib.pyplot as plt

In [40]:
import kagglehub
path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")
print("Path to dataset files:", path)

DATA_DIR = os.path.join(
    path,
    "PetImages"
)

Path to dataset files: C:\Users\ADMIN\.cache\kagglehub\datasets\bhavikjikadara\dog-and-cat-classification-dataset\versions\1


In [41]:
IMG_SIZE = 64
BATCH_SIZE = 128

In [42]:
# cai thien datagen
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

In [43]:
train_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

Found 20000 images belonging to 2 classes.
Found 4998 images belonging to 2 classes.


In [44]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # increase dropout
    Dense(2, activation='softmax')  # no activation here, use from_logits=True
])

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(2, activation="softmax") # Đảm bảo đầu ra là 2 lớp
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# huan luyen
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=["accuracy"]
)
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

model.summary()

Epoch 1/10
112/157 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.5727 - loss: 1.1955

In [55]:
model.save("PetImages.h5") # .h5 dung de load lai va du doan cho sau nay
print("Saved model")

Saved model


In [56]:
loss, accuracy = model.evaluate(val_data)
print(f"Accuracy: {accuracy:.4f}")
print(f"Loss: {loss:.4f}")

40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 354ms/step - accuracy: 0.5104 - loss: 0.6929
Accuracy: 0.5104
Loss: 0.6929


In [57]:
# load mo hinh
loaded_model = keras.models.load_model("PetImages.h5")
print("Loaded model from disk")

Loaded model from disk


In [58]:
# chuan bi label
class_names = list(train_data.class_indices.keys())
print(class_names)

['Cat', 'Dog']


In [59]:
# du doan
prediction = loaded_model.predict(img_array)
predicted_index = np.argmax(prediction)
predicted_label = class_names[predicted_index]

# print % cac lop du doan
print(prediction)
print(f"Predicted label: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
[[0.50658566 0.49341428]]
Predicted label: Cat
